#### Timing Iteration Alternatives

In [5]:
import time
def timer(func, *args):
    start = time.clock()
    for i in range(1000):
        func(*args)
    return time.clock() - start
timer(pow, 2, 1000)

/home/icpl13698/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/icpl13698/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


0.0012309999999999821

### New Timer Calls in 3.3

list comprehensions were nearly twice as fast as equivalent for loop
statements, and map was slightly quicker than list comprehensions when mapping a
function such as the abs (absolute value) built-in this way


On PyPy alone, list comprehensions beat map in this test, but the fact that all of PyPy’s
results are so much quicker today seems the larger point here. On CPython, map is still
quickest so far.

#### Timing Iterations and Pythons with timeit

The preceding section used homegrown timing functions to compare code speed. As
mentioned there, the standard library also ships with a module named timeit that can
be used in similar ways, but offers added flexibility and may better insulate clients from
some platform differences.

In [6]:
import timeit

In [12]:
timeit.timeit(stmt='[x ** 2 for x in range(1000)]', number=1000)     # Total time
                                           

0.22990375899826176

### Function gotchas

As you know, Python classifies names assigned in a function as locals by default; they
live in the function’s scope and exist only while the function is running. What you may
not realize is that Python detects locals statically, when it compiles the def ’s code, rather
than by noticing assignments as they happen at runtime. This leads to one of the most
common oddities posted on the Python newsgroup by beginners.
Normally, a name that isn’t assigned in a function is looked up in the enclosing module:

In [13]:
x=99
def select():                 # X used but not assigned
    print(x)                  # X found in global scope
    

In [14]:
select()

99


In [18]:
x=90
def select():                 
    print(x)
    x=88

In [19]:
select()

UnboundLocalError: local variable 'x' referenced before assignment

You get the name usage error shown here, but the reason is subtle. Python reads and
compiles this code when it’s typed interactively or imported from a module. While
compiling, Python sees the assignment to X and decides that X will be a local name
everywhere in the function. But when the function is actually run, because the assign-
ment hasn’t yet happened when the print executes, Python says you’re using an un-
defined name. According to its name rules, it should say this; the local X is used before
being assigned. In fact, any assignment in a function body makes a name local. Imports,
= , nested def s, nested classes, and so on are all susceptible to this behavior.
The problem occurs because assigned names are treated as locals everywhere in a func-
tion, not just after the statements where they’re assigned. Really, the previous example
is ambiguous: was the intention to print the global X and create a local X , or is this a
real programming error? Because Python treats X as a local everywhere, it’s seen as an
error; if you mean to print the global X , you need to declare it in a global statement:

In [28]:
def select():
    global x                      # Force X to be global (everywhere)
    print(x)
    x = 88

In [29]:
select()

90


Remember, though, that this means the assignment also changes the global X , not a
local X . Within a function, you can’t use both local and global versions of the same
simple name. If you really meant to print the global and then set a local of the same
name, you’d need to import the enclosing module and use module attribute notation
to get to the global version

In [32]:
x=90
def select():
    import __main__                        # Import enclosing module
    print(__main__.x)                      # Qualify to get to global version of name
    x = 88                              # Unqualified X classified as local
    print(x)                           # Prints local version of name

In [33]:
select()

90
88


Qualification (the .X part) fetches a value from a namespace object. The interactive
namespace is a module called __main__ , so __main__.X reaches the global version of X .
If that isn’t clear, check out Chapter 17.

##### Defaults and Mutable Objects

As noted briefly in Chapter 17 and Chapter 18, mutable values for default arguments
can retain state between calls, though this is often unexpected. In general, default ar-
gument values are evaluated and saved once when a def statement is run, not each time
the resulting function is later called. Internally, Python saves one object per default
argument attached to the function itself.
That’s usually what you want—because defaults are evaluated at def time, it lets you
save values from the enclosing scope, if needed (functions defined within loops by
factories may even depend on this behavior—see ahead). But because a default retains
an object between calls, you have to be careful about changing mutable defaults. For
instance, the following function uses an empty list as a default value, and then changes
it in place each time the function is called:

In [34]:
def saver(x=[]):                          # Saves away a list object
    x.append(1)                            # Changes same object each time!
    print(x)

In [35]:
saver([2])                               # Default not used

[2, 1]


In [36]:
saver([3])                                # Default not used

[3, 1]


In [37]:
saver()                                # Default used

[1]


In [38]:
saver()

[1, 1]


In [39]:
saver()

[1, 1, 1]


If that’s not the behavior you want, simply make a copy of the default at the start of
the function body, or move the default value expression into the function body. As long
as the value resides in code that’s actually executed each time the function runs, you’ll
get a new object each time through:

In [41]:
def saver(x=None):
    if x is None:                       # No argument passed?
        x = []                          # Run code to make a new list each time
    x.append(1)                         # Changes new list object
    print(x)

In [42]:
saver([2])

[2, 1]


In [43]:
saver()                               # Doesn't grow here

[1]


In [44]:
saver()

[1]


By the way, the if statement in this example could almost be replaced by the assignment
x = x or [] , which takes advantage of the fact that Python’s or returns one of its
operand objects: if no argument was passed, x would default to None , so the or would
return the new empty list on the right.
However, this isn’t exactly the same. If an empty list were passed in, the or expression
would cause the function to extend and return a newly created list, rather than ex-
tending and returning the passed-in list like the if version. (The expression becomes
[] or [] , which evaluates to the new empty list on the right; see the section “Truth
Tests” if you don’t recall why.) Real program requirements may call for either behavior.

Today, another way to achieve the value retention effect of mutable defaults in a pos-
sibly less confusing way is to use the function attributes we discussed in Chapter 19:

In [45]:
def saver():
    saver.x.append(1)
    print(saver.x)

In [46]:
saver.x=[]

In [47]:
saver()

[1]


In [48]:
saver.x=[3,4]

In [49]:
saver()

[3, 4, 1]


In [50]:
saver()

[3, 4, 1, 1]


In [51]:
saver()

[3, 4, 1, 1, 1]


In [52]:
saver.x

[3, 4, 1, 1, 1]

The function name is global to the function itself, but it need not be declared because
it isn’t changed directly within the function. This isn’t used in exactly the same way,
but when coded like this, the attachment of an object to the function is much more
explicit (and arguably less magical).

Functions such as this without a return are Python’s equivalent of what are called
“procedures” in some languages. They’re usually invoked as statements, and the None
results are ignored, as they do their business without computing a useful result.